<a href="https://colab.research.google.com/github/JADukeAIPI/AIPIProject/blob/main/dbpopulate_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import sqlite3 as db
import numpy as np
#Import .py modules with processed data: readcsv contains Kaggle data, WebScrapingAtlantaEvents contains Discover Atlanta event data,
#and APICALL contains Budget Car Rental data

import LuxuryCarAPIMultipleDates

In [ ]:
import APICALL

In [1]:
import pandas as pd
import sqlite3 as db
import numpy as np
import WebScrapingAtlantaEvents

In [ ]:
import readcsv

In [2]:
# Creates a blank db if one does not exist, otherwise opens connection to rentaldb, and drops the tables Kaggle, API, and Webscrape
conn = db.connect('rentaldb')
c = conn.cursor()
#c.execute("DROP TABLE IF EXISTS Kaggle")
#c.execute("DROP TABLE IF EXISTS API")
#c.execute("DROP TABLE IF EXISTS LuxuryAPI")
#c.execute("DROP TABLE IF EXISTS AllCarsAPI")
c.execute("DROP TABLE IF EXISTS Webscrape")
c.execute("DROP TABLE IF EXISTS LargeEvents")

In [3]:
#Runs the sort_lux_df function from the readcsv import. This function reads the CarRentalDataV1 csv data into a dataframe
#sorts it based on location (Atlanta), then filters the vehciles to only show luxury brands/models.
#csvdf = readcsv.sort_lux_df()
#This injects the sort_lux_df into the Kaggle table of the rentaldb 
#csvdf.to_sql('Kaggle', conn, if_exists='replace')

#Runs the get_events function from the WebScrapingAtlantaEvents import. This function scrapes Discover Atlanta, 
#and formats the events
webdf = WebScrapingAtlantaEvents.get_events('2022-11-19', '2023-03-03')
#This injects the get_events data into the Webscrape table of the rentaldb 
webdf.to_sql('Webscrape', conn, if_exists='replace')

#Gets large venue events at the Mercedes-Benz stadium and State Farm Arena, and injects into the LargeEvents table of the rentaldb
large_venue_query = 'Location.str.contains("State Farm Arena|Mercedes-Benz Stadium")'
web_large_venue_df = WebScrapingAtlantaEvents.get_events('2022-11-19', '2023-03-03', query_string=large_venue_query)
web_large_venue_df.to_sql('LargeEvents', conn, if_exists='replace')

#Runs the api_request function from APICALL. This function calls the API, and reads the data as JSON, then turns it into a df
#df2 = APICALL.api_request()
#Runs the clean_api_df function from APICALL. This function cleans the df2 dataframe that the api_request function created
#apidf = APICALL.clean_api_df(df2)
#This injects the cleaned API data into API table of the rentaldb
#apidf.to_sql('API', conn, if_exists='replace')

#Runs the get_bookings_multiple_dates function from LuxuryCarAPIMultipleDates. This function calls the API and requests the specified days of booking information.
#We will retrieve all cars and inject into AllCarsAPI, and luxury cars and inject into LuxuryAPI
#all_cars_apidf = LuxuryCarAPIMultipleDates.get_bookings_multiple_dates('2022-11-18', '2023-03-03')
#all_cars_apidf.to_sql('AllCarsAPI', conn, if_exists='replace')
#all_cars_apidf.to_csv('AllCarsAPI.csv')
#all_cars_apidf.to_pickle('AllCarsAPI.pkl')

#luxuryapidf = all_cars_apidf.loc[all_cars_apidf['Category']=='Luxury'] 
#luxuryapidf.to_sql('LuxuryAPI', conn, if_exists='replace')
#luxuryapidf.to_csv('LuxuryAPI.csv')
#luxuryapidf.to_pickle('LuxuryAPI.pkl')


32

In [8]:
#Verifying the tables are in the rentaldb
csvdf = pd.read_sql_query("SELECT * FROM Kaggle", conn)
apidf = pd.read_sql_query("SELECT * FROM API", conn)
luxuryapidf = pd.read_sql_query("SELECT * FROM LuxuryAPI", conn)
scrapedf = pd.read_sql_query("SELECT * FROM Webscrape", conn)
all_cars_apidf = pd.read_sql_query("SELECT * FROM AllCarsAPI", conn)
web_large_venue_df = pd.read_sql_query("SELECT * FROM LargeEvents", conn)

print('Kaggle Data')
display(csvdf.head())
print('API Data')
display(apidf.head())
print('All Car DAta')
display(all_cars_apidf.head())
print('Luxury API Data')
display(luxuryapidf.head(50))
print('Webscrape Data')
display(scrapedf.head())
print('Large Venue Events')
display(web_large_venue_df)

Kaggle Data


,index,fuelType,rating,renterTripsTaken,reviewCount,city,country,latitude,location.longitude,location.state,owner.id,daily_rate,make,model,type,year,airportcity
0,41,GASOLINE,5.00,12,10,Grayson,US,33.906905,-83.960234,GA,1295660,175,Mercedes-Benz,S-Class,car,2015,Atlanta
1,42,GASOLINE,5.00,2,2,Atlanta,US,33.891295,-84.453929,GA,12796797,200,BMW,7 Series,car,2018,Atlanta
2,50,GASOLINE,4.96,152,140,Atlanta,US,33.927684,-84.343055,GA,511103,155,Chevrolet,Corvette Stingray,car,2014,Atlanta
3,54,GASOLINE,5.00,23,21,Atlanta,US,33.740676,-84.358236,GA,8752492,209,Audi,RS 7,car,2014,Atlanta
4,57,ELECTRIC,4.88,18,16,Duluth,US,33.970512,-84.161462,GA,8609349,159,Tesla,Model S,car,2018,Atlanta


API Data


,index,Vehicle_id,Vehicle_Name,Category,Transmission,Dropoff_City,Base_Price,Pickup_City
0,221,673928006,Mercedes-Benz GLC,Luxury,Automatic,,367.62,
1,235,644881573,Cadillac XTS,Luxury,Automatic,,366.94,
2,216,636479888,Cadillac XTS,Luxury,Automatic,,353.16,
3,214,646119413,Cadillac XTS,Luxury,Automatic,,341.64,
4,179,723428946,BMW 5 Series,Luxury,Automatic,,338.18,


All Car DAta


...

,index,Vehicle_id,Vehicle_Name,Category,Transmission,Base_Price,Price,Date_Pickup,Date_Dropoff
0,82,644881573,Cadillac XTS,Luxury,Automatic,176.95,182.78,2022-11-18,2022-11-19
1,84,646119413,Cadillac XTS,Luxury,Automatic,163.90,169.30,2022-11-18,2022-11-19
2,93,644901168,Cadillac XTS,Luxury,Automatic,158.89,164.13,2022-11-18,2022-11-19
3,90,644861493,Cadillac XTS,Luxury,Automatic,146.87,151.71,2022-11-18,2022-11-19
4,89,646151293,Cadillac XTS,Luxury,Automatic,145.96,150.77,2022-11-18,2022-11-19
5,58,644841863,Cadillac XTS,Luxury,Automatic,134.82,139.26,2022-11-18,2022-11-19
6,80,646141193,Cadillac XTS,Luxury,Automatic,133.99,138.40,2022-11-18,2022-11-19
7,59,735898091,Cadillac XTS,Luxury,Automatic,133.99,138.40,2022-11-18,2022-11-19
8,57,735895261,Cadillac XTS,Luxury,Automatic,122.04,126.06,2022-11-18,2022-11-19
9,219,644881573,Cadillac XTS,Luxury,Automatic,155.11,160.22,2022-11-19,2022-11-20


Webscrape Data


,index,Event,Dates,Location,Description
0,1689,Champions Of Magic,2022-11-20,Cobb Energy Performing Arts Centre,Break away from the screen and experience live...
1,1690,Harry Potter: The Exhibition,2022-11-20,Purchase tickets online,Harry Potter: The Exhibition will make its sec...
2,1691,Retreat into Oneness,2022-11-20,Elohee Retreat Center,Retreat into Oneness offers a respite from the...
3,1692,PAW Patrol: Adventure Play,2022-11-20,Children’s Museum of Atlanta,Adventure Bay is coming to CMA! PAW Patrol: Ad...
4,1693,Free Birds–$ 2.00 Fall Flick at Aurora Cineplex,2022-11-20,Aurora Cineplex,FREE BIRDS ( PG)–our $ 2.00 Fall Family Flicks...


Large Venue Events


,index,Event,Dates,Location,Description
0,1700,Atlanta Falcons vs. Chicago Bears – Nov. 20,2022-11-20,Mercedes-Benz Stadium,Come watch the Atlanta Falcons take on the Chi...
1,1829,ATEEZ – The Fellowship: Break The Wall Tour,2022-11-22,State Farm Arena,K-pop giants ATEEZ have announced a monumental...
2,1865,Atlanta Hawks vs. Sacramento Kings,2022-11-23,State Farm Arena,Come cheer on the Hawks as they take on the Sa...
3,2080,Atlanta Hawks vs. Miami Heat,2022-11-27,State Farm Arena,Come cheer on the Hawks as they take on the Mi...
4,2264,Rod Wave – Beautiful Mind Tour,2022-12-01,State Farm Arena,Platinum-selling rapper Rod Wave has announced...
5,2298,Disney On Ice: Frozen & Encanto,2022-12-02,State Farm Arena,"For the first time, Disney On Ice invites fami..."
6,2315,Atlanta Hawks vs. Denver Nuggets,2022-12-02,State Farm Arena,Come cheer on the Hawks!
7,2361,2022 SEC Football Championship,2022-12-03,Mercedes-Benz Stadium,The 2022 SEC Championship Game is on course to...
8,2406,Atlanta Falcons vs. Pittsburgh Steelers – Dec. 4,2022-12-04,Mercedes-Benz Stadium,Come watch the Atlanta Falcons take on the Pit...
9,2430,Atlanta R&B Music Experience,2022-12-04,State Farm Arena,It was announced today that ‘Atlanta R&B Music...
